In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
#!pip install lifelines

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime, timedelta
import seaborn as sn
import time
import matplotlib.animation as animation
#from lifelines.plotting import plot_lifetimes
#from lifelines import KaplanMeierFitter
#kmf = KaplanMeierFitter()


In [12]:
df = pd.read_csv('/content/drive/MyDrive/Диплом/data_for_science (1).csv', sep=';')

In [13]:
df.at[13802, 'dis_date'] = '2020-06-13'
df.at[13802, 'get_date_at'] = '2020-06-13'
df.at[13802, 'receive_date_at'] = '2020-06-14'
df.at[13802, 'result_date_at'] = '2020-06-15'
df.at[13799, 'get_date_at' ] = '2020-06-18'
df.at[13799, 'receive_date_at'] = '2020-06-19'
df.at[13799, 'result_date_at'] = '2020-06-19'
df.at[13804, 'get_date_at'] = '2020-06-26'

In [14]:
df = df[df['get_date_at'].apply(lambda x: x[0]) == '2']

In [15]:
df = df[df['dis_date'].notna()]

In [16]:
df = df[df['dis_date'].apply(lambda x: x[0]) == '2']

In [17]:
#df.to_csv('/content/drive/MyDrive/Диплом/FixedDF.csv', index = False)

Запись состояний в таблицу

In [18]:
group = pd.read_excel('/content/drive/MyDrive/Диплом/Contact.xlsx')[['idx']]

In [19]:
df = df[df['idx'].isin(group['idx'])]

In [20]:
df = df[(df['mkb10_code'] == 'U07.2') | (df['mkb10_code'] == 'U07.1') | (df['mkb10_code'] == 'Z20.8')]

In [21]:
def condition_4(record):
  if ((record['mkb10_code'] == 'U07.2') & (record['IgM'] > 1)): return True;
  return False;

In [22]:
def condition_2(record):
  if ((record['mkb10_code'] == 'U07.1') | (record['IgM'] > 1)): return True;
  return False;

In [23]:
def condition_3(record):
  if (record['samples_result_id'] == '�� ����������'): return True;
  return False;

In [24]:
def condition_5(record):
  if ((record['IgG'] > 10) & (record['IgM'] < 1)): return True;
  return False;

In [25]:
def condition_6(record):
  if ((record['IgG'] < 10) & (record['IgM'] < 1)): return True;
  return False;

In [26]:
def condition_1(record):
  if (record['mkb10_code'] == 'Z20.8'): return True;
  return False;

In [27]:
def getState(index):
  record = df.loc[[index]].to_dict(orient = 'record')[0]
  #return record
  if (condition_1(record)): return 1;
  if (condition_6(record)): return 6;
  if (condition_5(record)): return 5;
  if (condition_4(record)): return 4;
  if (condition_2(record)): return 2;
  if (condition_3(record)): return 3;
  return 7;

VgetState = np.vectorize(getState);

In [28]:
# df['state'] = VgetState(df.index);

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


KeyboardInterrupt: ignored

In [ ]:
#df['state'].unique()

Дни достижения состояний

In [30]:

df['test_day'] = np.zeros(df.shape[0]).astype(int)

In [31]:
def SetTestNumber(idx):
    x = df[df['idx'] == idx]
    date_contact = min(x[x['mkb10_code']=='Z20.8']['dis_date'])
    df.loc[df['idx'] == idx, 'test_day'] = (pd.to_datetime(x['get_date_at']) - datetime.strptime(date_contact, '%Y-%m-%d'))
    # x['test_day'] = (pd.to_datetime(x['get_date_at']) - datetime.strptime(date_contact, '%Y-%m-%d'))

In [32]:

# for i in range(df['idx'].unique().size):
#     SetTestNumber(df['idx'].unique()[i])

KeyboardInterrupt: ignored


Состояние 7 создание

In [33]:

df = df[(pd.to_timedelta(df['test_day']) >= timedelta(days = 0)) & (pd.to_timedelta(df['test_day']) < timedelta(days = 40))]

In [34]:


df['test_day'] = pd.to_timedelta(df['test_day'])

In [35]:

def GetLast_test_day(idx):
    return max(df[df['idx'] == idx]['test_day']) + timedelta(days = 1)
    
VGetLast_test_day = np.vectorize(GetLast_test_day)

In [36]:

group = pd.DataFrame(df['idx'].unique(), columns = ['idx'])

In [37]:


# additional = pd.DataFrame([], columns = df.columns)
# additional['idx'] = group['idx']
# additional['state'] = 7
# additional['test_day'] = VGetLast_test_day(additional['idx'])
# additional

In [40]:


df = df.append(additional)

NameError: ignored

In [39]:

#df.to_csv('/content/drive/MyDrive/Диплом/dfWithStates.csv', index = False)

In [69]:
df = pd.read_csv('/content/drive/MyDrive/Диплом/dfWithStates.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [124]:
df[df['idx'] == 4570]

,idx,pd_id,person_id,ds_id,gender,birth_dt,mkb10_code,mkb10_name,dis_date,samples_type_id,get_date_at,receive_date_at,result_date_at,IgG,IgM,samples_result_id,mu_type,state,test_day
1986,4570,3181998.0,3100054.0,3358716.0,�,1935-01-07,U07.1,"�������������� ��������, ��������� ������� COV...",2020-05-17,�����/���������� �� ���������� � ����������,2020-06-18,2020-06-19,2020-06-19,NaN,NaN,�� ����������,��������� �����������,2,5 days
1987,4570,1350434.0,1247652.0,1424218.0,�,1935-01-07,Z20.8,������� � ������� � ����������� ��������� ����...,2020-06-13,�����/���������� �� ���������� � ����������,2020-06-13,2020-06-14,2020-06-15,NaN,NaN,�� ����������,��������,1,0 days
1988,4570,4049168.0,4000626.0,4286517.0,�,1935-01-07,U07.1,"�������������� ��������, ��������� ������� COV...",2020-06-14,"�����, ������� (���������)",2020-06-28,2020-06-28,2020-06-28,100.01,0.97,NaN,��������� ��������,5,15 days
1989,4570,3896074.0,3842295.0,4122859.0,�,1935-01-07,U07.1,"�������������� ��������, ��������� ������� COV...",2020-06-14,"�����, ������� (���������)",2020-06-26,2020-06-26,2020-06-26,133.32,0.97,NaN,��������� ��������,5,13 days
1990,4570,3732973.0,3671253.0,3950395.0,�,1935-01-07,U07.1,"�������������� ��������, ��������� ������� COV...",2020-06-14,"�����, ������� (���������)",2020-06-22,2020-06-22,2020-06-22,145.68,1.01,NaN,��������� ��������,2,9 days
1991,4570,3921097.0,3868222.0,4149103.0,�,1935-01-07,U07.1,"�������������� ��������, ��������� ������� COV...",2020-06-14,�����/���������� �� ���������� � ����������,2020-06-25,2020-06-26,2020-06-27,NaN,NaN,�� ����������,��������� ��������,2,12 days
337647,4570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,16 days


Заполнение таблицы

In [160]:
Table = pd.DataFrame([])
Table['idx'] = group['idx']
Table = Table.set_index('idx')
for i in range(0,41):
  Table[i] = np.ones(Table.shape[0]).astype(int)

In [161]:
def fill_patient(idx):
    patient = df[df['idx'] == idx][['test_day','state']]
    patient['test_day'] = patient['test_day'].apply(lambda x: int(x.split(' ')[0]))
    timeline = np.ones(Table.shape[1])
    for i in range(patient.shape[0]):
        timeline[patient['test_day'].values[i]] = int(patient['state'].values[i])
    for i in range(1, timeline.size):
        timeline[i] = max(timeline[i-1], timeline[i])
    return timeline.astype(int)

Vfill_patient = np.vectorize(fill_patient)

In [162]:
for i in Table.index:
    Table.loc[i] = fill_patient(i)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [163]:
Table.to_csv('/content/drive/MyDrive/Диплом/Table.csv')

In [190]:
Table = pd.read_csv('/content/drive/MyDrive/Диплом/Table.csv')
Table = Table.set_index('idx')

In [191]:
Table[Table.index == 4570]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
idx,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4570,1,1,1,1,1,2,2,2,2,2,2,2,2,5,5,5,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7


In [189]:
Table = Table([(np.array_equal(Table.loc[i].unique(), [1,7]) != True) for i in Table.index])

TypeError: ignored

In [110]:
def getpk(i, j, k):
    g1 = Table[Table[str(k)] == i]
    g2 = g1[g1[str(k+1)] == j]
    if g1.shape[0] == 0:
        return 0
    return (g2.shape[0] / g1.shape[0])

In [111]:
def getp(i, j):
    p = np.zeros(Table.shape[1])
    for k in range(p.size - 1):
        p[k] = getpk(i, j, k)
    return p.mean()

In [112]:
getp(1,7)

0.11409756579567965

In [85]:
P = np.zeros((7,7))

In [86]:

for i in range(1,8):
    for j in range(1,8):
        print (i,j, getp(i,j))
        P[i-1,j-1] = getp(i,j)

KeyError: ignored

In [87]:
P

array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])

In [54]:
pd.DataFrame(np.around(P, 5), columns = np.linspace(1,7,7).astype(int), index = np.linspace(1,7,7).astype(int))

,1,2,3,4,5,6,7
1,0.95583,0.00144,0.00020,0.00010,0.00006,0.00010,0.01789
2,0.00000,0.75961,0.00191,0.00098,0.00254,0.00149,0.20908
3,0.00000,0.00000,0.73505,0.00987,0.00109,0.00145,0.22814
4,0.00000,0.00000,0.00000,0.79819,0.00089,0.00048,0.17605
5,0.00000,0.00000,0.00000,0.00000,0.76442,0.00033,0.21086
6,0.00000,0.00000,0.00000,0.00000,0.00000,0.79484,0.18076
7,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.97561


In [52]:
n = 40
h = np.zeros((n, 7))
h[0][0] = 10000
for j in range(1,n):
    for i in range(7):
        h[j] = h[j] + h[j-1][i] * P[i]

In [ ]:

h

In [ ]:

plt.rcParams["figure.figsize"] = (3.2 * 4, 2.4 * 10)
for i in range(n):
    plt.subplot(10, 4, (i + 1))
    plt.bar(np.linspace(1,6,6).astype(int), h[i][:-1])
plt.show()
plt.rcParams["figure.figsize"] = (6.4 , 4.8 )

Анимация

In [ ]:


from IPython.display import HTML
from matplotlib.animation import FuncAnimation, PillowWriter  


x = np.linspace(1, 7, 7).astype(int)
n = h.shape[1]

def drawAnimation(x, a):
    fig, ax = plt.subplots()  
    l, = plt.bar(x, h[0])
    # plt.plot([], [], '-')  

    def init():
        ax.set_xlim(min(x), max(x))
        ax.set_ylim(np.min(a), np.max(a))

    def update(i):  
        l.set_data(x, a[int(i)])

    ani = FuncAnimation(fig, update, np.linspace(0, a.shape[0] - 1, a.shape[0]), init_func=init)
    plt.show()

    writer = PillowWriter(fps=25)  
    return ani
    #ani.save("bars.gif", writer=writer) 

ani = drawAnimation(x, h) 
HTML(ani.to_jshtml())

Каплан-Майер

In [ ]:


def prob( i, k):
    # p = n_out/n_in
    g1 = Table[Table[k] == i]
    g2 = g1[g1[k+1] == i]
    g3 = Table[Table[k] == j]
    g4 = g1[g1[k+1] == j]
    if g1.shape[0] == 0:
        return 0
    return ((g2.shape[0]-g1.shape[0] )/ (g4.shape[0]-g3.shape[0]))


In [ ]:


plt.bar([prob(1, 7, i) for i in range(1, 41)])